In [1]:
# 4. Clustering:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist, pdist, squareform

# Custom K-Means Implementation
def kmeans_custom(data, n_clusters, max_iters=100, tol=1e-4):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, n_clusters, replace=False)]
    prev_centroids = centroids.copy()

    for _ in range(max_iters):
        # Assign labels based on closest centroid
        distances = cdist(data, centroids, 'euclidean')
        labels = np.argmin(distances, axis=1)

        # Compute new centroids
        new_centroids = np.array([data[labels == i].mean(axis=0) for i in range(n_clusters)])

        # Check convergence
        if np.all(np.abs(new_centroids - prev_centroids) < tol):
            break
        prev_centroids = new_centroids

    return labels, new_centroids

# Custom Expectation-Maximization (EM) Clustering Implementation (Gaussian Mixture Model)
def em_custom(data, n_components, max_iters=100, tol=1e-4):
    n_samples, n_features = data.shape
    # Initialize the parameters (random initialization)
    means = data[np.random.choice(n_samples, n_components, replace=False)]
    covariances = np.array([np.cov(data.T)] * n_components)
    weights = np.ones(n_components) / n_components
    prev_log_likelihood = None

    for _ in range(max_iters):
        # E-step: calculate responsibilities (posterior probabilities)
        responsibilities = np.array([weights[k] * multivariate_gaussian(data, means[k], covariances[k]) for k in range(n_components)])
        responsibilities /= responsibilities.sum(axis=0)

        # M-step: update parameters based on responsibilities
        Nk = responsibilities.sum(axis=1)
        means = (responsibilities @ data) / Nk[:, np.newaxis]
        covariances = np.array([np.cov(data.T, aweights=responsibilities[k]) for k in range(n_components)])
        weights = Nk / n_samples

        # Calculate log likelihood
        log_likelihood = np.sum(np.log(np.sum(responsibilities, axis=0)))
        if prev_log_likelihood is not None and np.abs(log_likelihood - prev_log_likelihood) < tol:
            break
        prev_log_likelihood = log_likelihood

    labels = np.argmax(responsibilities, axis=0)
    return labels, means, covariances, weights

# Helper function for multivariate Gaussian distribution
def multivariate_gaussian(data, mean, cov):
    n_features = data.shape[1]
    diff = data - mean
    return np.exp(-0.5 * np.sum(np.dot(diff, np.linalg.inv(cov)) * diff, axis=1)) / np.sqrt((2 * np.pi) ** n_features * np.linalg.det(cov))

# Custom K-Medoids Implementation
def kmedoids_custom(data, n_clusters, max_iters=100):
    n_samples = data.shape[0]
    # Randomly initialize medoids (choosing n_clusters data points)
    medoids = data[np.random.choice(n_samples, n_clusters, replace=False)]
    prev_medoids = medoids.copy()

    for _ in range(max_iters):
        # Assign labels based on closest medoid
        distances = cdist(data, medoids, 'euclidean')
        labels = np.argmin(distances, axis=1)

        # Recalculate medoids
        new_medoids = np.array([data[labels == i].mean(axis=0) for i in range(n_clusters)])
        new_medoids = new_medoids.round().astype(int)  # Round to nearest point for K-medoids

        # Check convergence
        if np.all(np.abs(new_medoids - prev_medoids) == 0):
            break
        prev_medoids = new_medoids

    return labels, new_medoids

# Compute metrics
def compute_sse(data, labels, centroids):
    """Compute Sum of Squared Errors (SSE)."""
    # Compute pairwise distances from data points to centroids
    distances = cdist(data, centroids, 'euclidean')
    
    # Find the minimum distance (for each point) and square it
    sse = np.sum(np.min(distances ** 2, axis=1))
    
    return sse

def compute_silhouette(data, labels):
    """Compute Silhouette Score."""
    return silhouette_score(data, labels) if len(np.unique(labels)) > 1 else np.nan

def compute_beta_cv(data, labels):
    """Compute BetaCV."""
    clusters = np.unique(labels)
    cluster_means = np.array([data[labels == cluster].mean(axis=0) for cluster in clusters])
    overall_mean = data.mean(axis=0)

    inter_cluster_var = np.sum([len(data[labels == cluster]) * np.sum((mean - overall_mean) ** 2) for cluster, mean in enumerate(cluster_means)])
    intra_cluster_var = np.sum([np.sum((data[labels == cluster] - mean) ** 2) for cluster, mean in enumerate(cluster_means)])

    return inter_cluster_var / intra_cluster_var

def compute_dunn_index(data, labels):
    """Compute Dunn Index."""
    clusters = np.unique(labels)
    distances = cdist(data, data)
    min_intercluster_dist = np.inf
    max_intracluster_dist = 0

    for i in clusters:
        cluster_points = np.where(labels == i)[0]
        intracluster_dist = np.max(distances[np.ix_(cluster_points, cluster_points)])
        max_intracluster_dist = max(max_intracluster_dist, intracluster_dist)

        for j in clusters:
            if i != j:
                other_cluster_points = np.where(labels == j)[0]
                intercluster_dist = np.min(distances[np.ix_(cluster_points, other_cluster_points)])
                min_intercluster_dist = min(min_intercluster_dist, intercluster_dist)

    return min_intercluster_dist / max_intracluster_dist

def compute_huberts_statistic(data, labels):
    """Compute Hubert’s Statistic."""
    distance_matrix = pdist(data, metric='euclidean')
    cluster_matrix = np.equal.outer(labels, labels).astype(int)
    flat_distances = squareform(distance_matrix)
    return np.corrcoef(flat_distances.flatten(), cluster_matrix.flatten())[0, 1]

# Load dataset
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# Remove categorical attributes and target variable
numerical_data = data.select_dtypes(include=['float64', 'int64'])
target_variable = 'temp'

# Fill missing values with mode
for column in numerical_data.columns:
    mode_value = numerical_data[column].mode()[0]
    numerical_data[column] = numerical_data[column].fillna(mode_value)

X_original = numerical_data.drop(columns=[target_variable]).values

# Apply Z-score transformation
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X_original)

# Number of clusters
k = 3

# K-Means clustering
kmeans_original_labels, kmeans_original_centroids = kmeans_custom(X_original, k)
kmeans_transformed_labels, kmeans_transformed_centroids = kmeans_custom(X_transformed, k)

# EM Clustering
em_original_labels, em_original_means, em_original_covariances, em_original_weights = em_custom(X_original, k)
em_transformed_labels, em_transformed_means, em_transformed_covariances, em_transformed_weights = em_custom(X_transformed, k)

# K-Medoids clustering
kmedoids_original_labels, kmedoids_original_centroids = kmedoids_custom(X_original, k)
kmedoids_transformed_labels, kmedoids_transformed_centroids = kmedoids_custom(X_transformed, k)

# Prepare results
clustering_results = [
    ("Original K-Means", X_original, kmeans_original_labels, kmeans_original_centroids),
    ("Transformed K-Means", X_transformed, kmeans_transformed_labels, kmeans_transformed_centroids),
    ("Original EM", X_original, em_original_labels, em_original_means),
    ("Transformed EM", X_transformed, em_transformed_labels, em_transformed_means),
    ("Original K-Medoids", X_original, kmedoids_original_labels, kmedoids_original_centroids),
    ("Transformed K-Medoids", X_transformed, kmedoids_transformed_labels, kmedoids_transformed_centroids),
]

# Compute evaluation metrics
for name, data, labels, centroids in clustering_results:
    sse = compute_sse(data, labels, centroids) if centroids is not None else "N/A"
    silhouette = compute_silhouette(data, labels)
    beta_cv = compute_beta_cv(data, labels)
    dunn_index = compute_dunn_index(data, labels)
    huberts_statistic = compute_huberts_statistic(data, labels)

    print(f"{name} Results:")
    print(f"  SSE: {sse}")
    print(f"  Silhouette Score: {silhouette}")
    print(f"  Dunn Index: {dunn_index}")
    print(f"  BetaCV: {beta_cv}")
    print(f"  Hubert’s Statistic: {huberts_statistic}")
    print("\n")

Original K-Means Results:
  SSE: 41038704.06121276
  Silhouette Score: 0.18035134956090837
  Dunn Index: 0.015027991559122982
  BetaCV: 0.5203663740886495
  Hubert’s Statistic: -0.37318867525848504


Transformed K-Means Results:
  SSE: 53861.94751656399
  Silhouette Score: 0.16326744911533145
  Dunn Index: 0.026381297333790715
  BetaCV: 0.35183007514812903
  Hubert’s Statistic: -0.34452132007840824


Original EM Results:
  SSE: 49455517.48023911
  Silhouette Score: 0.02612281443633195
  Dunn Index: 0.011765458201096927
  BetaCV: 0.17711686580366184
  Hubert’s Statistic: -0.1346768898545876


Transformed EM Results:
  SSE: 60161.67663319381
  Silhouette Score: 0.034827172290873794
  Dunn Index: 0.018652505359670838
  BetaCV: 0.09133128520935428
  Hubert’s Statistic: -0.12088872476640357


Original K-Medoids Results:
  SSE: 38965282.268565
  Silhouette Score: 0.2429475277793729
  Dunn Index: 0.026366396730414538
  BetaCV: 0.806993251578068
  Hubert’s Statistic: -0.48055768329421783


Tra